# <center>Peticos 🐶😺❤️</center>
### Dados do Banco do 1° -> Banco do 2°

Imports

In [18]:
import psycopg2 as pg
import os
from dotenv import load_dotenv
from datetime import datetime
import numpy
from openpyxl import Workbook, load_workbook

Pegando a senha do banco no .env

In [19]:
#Pegando a senha do banco:
load_dotenv()
password = os.getenv('PASSWORD12')

Conexão com o Banco do 1°

In [20]:
#Conectando com o banco:
conn1 = pg.connect(
    dbname = "dbPeticos_1ano",
    user = "avnadmin",
    password = password,
    host = "db-peticos-cardosogih.k.aivencloud.com",
    port = 16207
)
cursor1 = conn1.cursor()

Conexão com o Banco do 2°

In [21]:
#Conectando com o banco:
conn2 = pg.connect(
    dbname = "dbPeticos_2ano",
    user = "avnadmin",
    password = password,
    host = "db-peticos-cardosogih.k.aivencloud.com",
    port = 16207
)
cursor2 = conn2.cursor()

#### Funções:

Função para verificar se aconteceu alguma mudança no banco nos últimos 5 min:

In [34]:
try:
    cursor1.execute(
        '''
            SELECT column_name, data_type
            FROM information_schema.columns
            WHERE table_name = 'log_total_1';
        '''
)
    modification_times = cursor1.fetchall()
except Exception as e:
    cursor1.execute("ROLLBACK;")
    print("Error occurred:", e)

In [35]:
modification_times

[('alteration_date', 'timestamp without time zone'),
 ('id', 'integer'),
 ('table_name', 'character varying'),
 ('operation', 'character varying'),
 ('user_', 'character varying')]

In [48]:
#Verificando a diferença de tempo entre a data da modificação e a data de hoje:
def diff_5min(data_hora, hoje):
    #Formatando novamente as datas:
    data_hora = datetime.strptime(data_hora, '%d/%m/%Y %H:%M:%S')
    hoje = datetime.strptime(hoje, '%d/%m/%Y %H:%M:%S')

    #Pegando a diferença em minutos:
    diff = (hoje - data_hora).total_seconds() // 60

    #Caso a diferença for menor ou igual a 5 min, retorna 1:
    if diff <= 5 and diff > 0:
        return 1
    
    #Caso não há nenhuma mudança nos últimos 5 min retorna 0:
    return 0

### Funções de CRUD:

Insert:

In [ ]:
#Função para passar chamar as procedures de insert correta de acordo com a tabela que precisa ter inserção:
def insert(table_name_1, id_1, table_name_2, id):
    #Explicando cada um dos parametros que devem ser passados:
    '''
    - table_name_1 = nome da tabela do primeiro ano
    - id_1 = nome da coluna do id/pk na tabela do primeiro ano
    - table_name_2 = nome da tabela do segundo ano
    - id = id que foi inserido na tabela do primeiro ano
    '''
    #Trazendo o valor que precisa ser inserido de acordo com um certo id:
    cursor1.execute("SELECT * FROM %s WHERE %s = %s", (table_name_1, id_1, id))
    tabela = cursor1.fetchall()

    #Verificando em qual tabela deve ocorrer o insert:
    if table_name_2 == 'admin':
        #Inserindo os novos dados na tabela de admin:
        cursor2.execute("INSERT INTO admin (id, name, email, password) VALUES(%s, %s, %s, %s)", (tabela[0], tabela[1], tabela[2], tabela[3]))
    elif table_name_2 == '':
        cursor2.execute('')


Update: 

In [51]:
#Função para chamar as procedures de update corretas para cada tabela:
def insert(table_name_1, id_1, table_name_2, id_2, id):
    #Explicando cada um dos parametros que devem ser passados:
    '''
    - table_name_1 = nome da tabela do primeiro ano
    - id_1 = nome da coluna do id/pk na tabela do primeiro ano
    - table_name_2 = nome da tabela do segundo ano
    - id_2 = nome da coluna de id/pk na tabela do segundo ano
    - id = id que foi alterado na tabela do primeiro ano
    '''
    #Trazendo o valor que precisa ser inserido de acordo com um certo id:
    cursor1.execute("SELECT * FROM admin.administradores WHERE %s = %s", (id_1, id))
    tabela = cursor1.fetchall()

    #Verificando em qual tabela deve ocorrer o insert:
    if table_name_2 == 'admin':
        #Inserindo os novos dados na tabela de admin:
        cursor2.execute("UPDATE admin.administradores SET name = %s, email = %s, password = %s WHERE %s = %s", (tabela[1], tabela[2], tabela[3], id_2, id))
    elif table_name_2 == '':
        cursor2.execute('')

Delete:

#### Pegando os dados do banco do 1° ano e atualizando no banco do 2° ano:

In [ ]:
#Nome das tabelas e ids/pks do banco do 1° ano:
dict_tabelas_1 = {'administradores': 'id',
                'dica_do_dia': 'id_dica',
                'endereco': 'id_endereco',
                'local': 'id_local',
                'telefone_local': 'id_telefone_local',
                'raca': 'id_raca',
                'especie': 'id_especie'}

#Nomes das tabelas e ids/pks do banco do 2° ano:
dict_tabelas_2 = {'admin': 'id_admin',
                'day_hint': 'id_hint',
                'address': 'id_address',
                'locations': 'id_local',
                'local_phone': 'id_local_phone',
                'race': 'id_race',
                'specie': 'id_specie'}

#Armazenando a data atual, com a formatação correta:
hoje = datetime.now().strftime('%d/%m/%Y %H:%M:%S')

#Indice para verificar as chaves e valores do dicionário das tabelas e ids/pks do segundo ano:
index = 0

#For para verificar se houve modificação em alguma das tabelas da lista "tabelas_1":
for i in dict_tabelas_1:
    #Pegando o campo de hora para verificar se ocorreu alguma modificação nos últimos 5 minutos em um tabela especifica:
    cursor1.execute("SELECT alteration_date FROM log_total_1 WHERE table_name = %s", (i, ))
    data_hora = cursor1.fetchall()

    #Verificando se a tabela está vazia:
    if data_hora != []:
        for d in data_hora:
            #Formatando a data:
            data_hora_formatada = d[0].strftime('%d/%m/%Y %H:%M:%S')
            
            #Utilizando função diff_5min para verificar se ocorreu alguma mudança no banco nos últimos 5 minutos, caso ocorra alguma mudança a função retornará 1, caso não haja, retornará 0:
            diff = diff_5min(data_hora_formatada, hoje)

            #Verificando se o retorno da função foi 1 ou 0:
            #Caso o retorno tenha sido 1 as funções de insert, update ou delete, serão chamadas de acordo com a operação que foi feita nessa linha:
            if diff == 1:
                #Verificando qual foi a operação feita e o id que foi adicionado, modificado ou excluído:
                cursor1.execute("SELECT operation, id FROM log_total_1")
                operacao = cursor1.fetchall()

                #Verificanso qual operação foi feita
                if operacao[0] == 'INSERT':
                    #Chamando a função de insert:
                    insert(i, dict_tabelas_1[i], dict_tabelas_2.keys()[index], id)
                # elif operacao[0] == 'DELETE':
                #     #Chamando a função de delete:
                #     delete(tabelas_2[j], id)
                # else:
                #     #Chamando a função de update:
                #     update(i, tabelas_2[j]
            else:
                #Apresentando a mensagem caso não haja o que modificar:
                print('Nenhuma mudança aconteceu nos últimos 5 minutos')
        
            #Adicionando +1 no indice:
            index+=1
    else:
        #Apresentando a mensagem caso a tabela esteja vazia:
        print('Não há nenhuma dado dessa tabela!')

administradores
Nenhuma mudança aconteceu nos últimos 5 minutos
Nenhuma mudança aconteceu nos últimos 5 minutos
Nenhuma mudança aconteceu nos últimos 5 minutos
Nenhuma mudança aconteceu nos últimos 5 minutos
Nenhuma mudança aconteceu nos últimos 5 minutos
Nenhuma mudança aconteceu nos últimos 5 minutos
dica_do_dia
Nenhuma mudança aconteceu nos últimos 5 minutos
Nenhuma mudança aconteceu nos últimos 5 minutos
Nenhuma mudança aconteceu nos últimos 5 minutos
Nenhuma mudança aconteceu nos últimos 5 minutos
endereco
Não há nenhuma dado dessa tabela!
local
Nenhuma mudança aconteceu nos últimos 5 minutos
Nenhuma mudança aconteceu nos últimos 5 minutos
Nenhuma mudança aconteceu nos últimos 5 minutos
Nenhuma mudança aconteceu nos últimos 5 minutos
Nenhuma mudança aconteceu nos últimos 5 minutos
Nenhuma mudança aconteceu nos últimos 5 minutos
Nenhuma mudança aconteceu nos últimos 5 minutos
Nenhuma mudança aconteceu nos últimos 5 minutos
Nenhuma mudança aconteceu nos últimos 5 minutos
Nenhuma mud

In [1]:
ids_1 = {'administradores': 'id', 'dica_do_dia': 'id_dica', 'endereco': 'id_endereco', 'local': 'id_local', 'telefone_local': 'id_telefone_local', 'raca': 'id_raca', 'especie': 'id_especie'}
ids_2 = {'admin': 'id_admin', 'day_hint': 'id_hint', 'address': 'id_address', 'locations': 'id_local', 'local_phone': 'id_local_phone', 'race': 'id_race', 'specie': 'id_specie'}

In [10]:
list(ids_1.keys())

['administradores',
 'dica_do_dia',
 'endereco',
 'local',
 'telefone_local',
 'raca',
 'especie']

In [11]:
dicio = {}
for i in range(len(ids_1.keys())):
    dicio[list(ids_1.keys())[i]] = list(ids_2.keys())[i]
dicio

{'administradores': 'admin',
 'dica_do_dia': 'day_hint',
 'endereco': 'address',
 'local': 'locations',
 'telefone_local': 'local_phone',
 'raca': 'race',
 'especie': 'specie'}

In [ ]:
# cursor1.execute("SELECT * FROM local ORDER BY id_local")
# tabela = cursor1.fetchall()

# print(tabela)

# #print(tabela)
# try:
#     for i in tabela:
#         cursor2.execute("INSERT INTO locations (id_admin, name, email, password) VALUES(%s, %s, %s, %s)", (i[0],i[1], i[2], i[3]))
#         conn2.commit()

# except Exception as e:
#     print(f"Ocorreu um erro: {i}", e)
#     conn2.rollback()

In [ ]:
# name_excel = '../BI/Dados/Endereco.xlsx'

# #Verificando se existe o excel existe:
# if os.path.exists(name_excel):
#     # Carrega um arquivo Excel:
#     workbook = load_workbook(name_excel)

#     # Abrindo uma planilha Excel:
#     excel = workbook.active
#     excel = workbook.create_sheet('Endereço')

#     #Adicionando mais valores ao excel:
#     for i in tabela:
#         add = [i[0], i[1], i[2], i[3]]
#         excel.append(add)

#     #Salvando o arquivo:
#     workbook.save(name_excel)
# else:
#     #Adicionando mais valores ao excel:
#     for i in tabela:
#         add_dolar = [i[0], i[1], i[2], i[3]]